# Create a buffer for coastal wetland 


In this part to feed the coastal wetalnds shapefile the ones that  we remove the ones overlap with or stream watersheds we need to create buffer for them to be able to catch all the indirect nutrient load 
therefore I create a 50 meter buffer around the coastalwetlands and remoeb the overlapping buffers

In [1]:
import os 
import arcpy
from arcpy import management as DM
# Set environment
arcpy.env.overwriteOutput = True

In [2]:
# Input and output paths
outpath = r"D:\Users\abolmaal\Arcgis\NASAOceanProject\GIS_layer\CoastalWatersheds"
outErase_Riper = os.path.join(outpath, "Erase_Riperian")
out_dir = os.path.join(outpath, "Buff_CW")

# Ensure output folder exists
os.makedirs(out_dir, exist_ok=True)

# Buffer distance
buffer_distance = "50 Meters"
id_field = "CW_Id"
fields_to_preserve = ["CW_Id", "ID_Coastal","Wetland","start_lat","start_lon","CW_Area"]  # Add more if needed

# Input shapefiles
erase_buffers = {
    "avg": os.path.join(outErase_Riper, "Wetland_connected_avg_erasebuff_50.shp"),
    "high": os.path.join(outErase_Riper, "Wetland_connected_high_erasebuff_50.shp"),
    "low": os.path.join(outErase_Riper, "Wetland_connected_low_erasebuff_50.shp"),
    "surge": os.path.join(outErase_Riper, "Wetland_connected_surge_erasebuff_50.shp")
}

# Output shapefiles
buffer_fc = {
    "avg": os.path.join(out_dir, "Wetland_connected_avg_erasebuff_50_buff50.shp"),
    "high": os.path.join(out_dir, "Wetland_connected_high_erasebuff_50_buff50.shp"),
    "low": os.path.join(out_dir, "Wetland_connected_low_erasebuff_50_buff50.shp"),
    "surge": os.path.join(out_dir, "Wetland_connected_surge_erasebuff_50_buff50.shp")
}

no_overlap_fc = {
    "avg": os.path.join(out_dir, "Wetland_connected_avg_erasebuff_50_buff50_removeoverlap.shp"),
    "high": os.path.join(out_dir, "Wetland_connected_high_erasebuff_50_buff50_removeoverlap.shp"),
    "low": os.path.join(out_dir, "Wetland_connected_low_erasebuff_50_buff50_removeoverlap.shp"),
    "surge": os.path.join(out_dir, "Wetland_connected_surge_erasebuff_50_buff50_removeoverlap.shp")
}

arcpy.env.overwriteOutput = True

In [3]:
# Loop through each buffer
for key in erase_buffers:
    print(f"\n▶ Processing {key.upper()} buffers...")

    in_fc = erase_buffers[key]
    buffer_out = buffer_fc[key]
    final_out = no_overlap_fc[key]

    # Step 1: Create buffer (with ORIG_FID to track source)
    print("🟦 Creating buffer...")
    arcpy.Buffer_analysis(
        in_features=in_fc,
        out_feature_class=buffer_out,
        buffer_distance_or_field=buffer_distance,
        line_side="FULL",
        line_end_type="ROUND",
        dissolve_option="NONE"
    )

    # Step 2: Copy buffer schema to final output
    print("📂 Creating final output feature class...")
    arcpy.CreateFeatureclass_management(
        out_path=os.path.dirname(final_out),
        out_name=os.path.basename(final_out),
        geometry_type="POLYGON",
        template=buffer_out
    )

    # Step 3: Get list of fields to preserve
    field_list = [f.name for f in arcpy.ListFields(buffer_out) if f.type not in ("OID", "Geometry")]
    field_list.append("SHAPE@")

    # Step 4: Get field type for SQL syntax
    field_type = [f.type for f in arcpy.ListFields(buffer_out) if f.name == id_field][0]

    # Step 5: Loop through features
    with arcpy.da.SearchCursor(buffer_out, field_list) as cursor:
        for row in cursor:
            attr_values = {field: row[i] for i, field in enumerate(field_list) if field != "SHAPE@"}
            geom = row[-1]

            # SQL clause to exclude current polygon
            if field_type in ['Integer', 'SmallInteger', 'Double']:
                where_clause = f"{id_field} <> {attr_values[id_field]}"
                self_clause = f"{id_field} = {attr_values[id_field]}"
            else:
                where_clause = f"{id_field} <> '{attr_values[id_field]}'"
                self_clause = f"{id_field} = '{attr_values[id_field]}'"

            # Select current and others
            arcpy.MakeFeatureLayer_management(buffer_out, "others", where_clause)
            arcpy.MakeFeatureLayer_management(buffer_out, "self", self_clause)

            # Copy current geometry
            temp_self = os.path.join("in_memory", f"self_{attr_values[id_field]}")
            arcpy.CopyFeatures_management("self", temp_self)

            # Erase overlap
            temp_result = os.path.join("in_memory", f"erase_{attr_values[id_field]}")
            arcpy.Erase_analysis(temp_self, "others", temp_result)

            # Add attributes back
            with arcpy.da.UpdateCursor(temp_result, field_list) as update_cursor:
                for urow in update_cursor:
                    for i, field in enumerate(field_list[:-1]):
                        urow[i] = attr_values[field]
                    update_cursor.updateRow(urow)

            # Append result
            arcpy.Append_management(temp_result, final_out, "NO_TEST")

            # Clean up memory
            for tmp in [temp_self, temp_result, "self", "others"]:
                if arcpy.Exists(tmp):
                    arcpy.Delete_management(tmp)

    print(f"✅ Done: {final_out}")


▶ Processing AVG buffers...
🟦 Creating buffer...
📂 Creating final output feature class...
✅ Done: D:\Users\abolmaal\Arcgis\NASAOceanProject\GIS_layer\CoastalWatersheds\Buff_CW\Wetland_connected_avg_erasebuff_50_buff50_removeoverlap.shp

▶ Processing HIGH buffers...
🟦 Creating buffer...
📂 Creating final output feature class...
✅ Done: D:\Users\abolmaal\Arcgis\NASAOceanProject\GIS_layer\CoastalWatersheds\Buff_CW\Wetland_connected_high_erasebuff_50_buff50_removeoverlap.shp

▶ Processing LOW buffers...
🟦 Creating buffer...
📂 Creating final output feature class...
✅ Done: D:\Users\abolmaal\Arcgis\NASAOceanProject\GIS_layer\CoastalWatersheds\Buff_CW\Wetland_connected_low_erasebuff_50_buff50_removeoverlap.shp

▶ Processing SURGE buffers...
🟦 Creating buffer...
📂 Creating final output feature class...
✅ Done: D:\Users\abolmaal\Arcgis\NASAOceanProject\GIS_layer\CoastalWatersheds\Buff_CW\Wetland_connected_surge_erasebuff_50_buff50_removeoverlap.shp


In [ ]:
# Fields to check and correct (lowercase key : (correct name, field type))
required_fields = {
    'cw_id': ('CW_Id', 'LONG'),
    'cw_area': ('CW_Area', 'DOUBLE')
}

for label, shapefile in erase_buffers.items():
    print(f"\n🔍 Checking shapefile: {shapefile}")
    
    fields = arcpy.ListFields(shapefile)
    field_names = [field.name for field in fields]
    field_names_lower = [name.lower() for name in field_names]

    print("📋 Existing Fields:", field_names)

    for lowercase_key, (correct_name, field_type) in required_fields.items():
        matched = None
        for field in fields:
            if field.name.lower() == lowercase_key:
                matched = field.name
                break

        if matched:
            if matched != correct_name:
                if correct_name in field_names:
                    print(f"⚠️ '{correct_name}' already exists — skipping rename from '{matched}'")
                else:
                    print(f"✏️ Renaming '{matched}' to '{correct_name}' using workaround")

                    # Step 1: Add new field
                    arcpy.AddField_management(shapefile, correct_name, field_type)

                    # Step 2: Copy values
                    with arcpy.da.UpdateCursor(shapefile, [matched, correct_name]) as cursor:
                        for row in cursor:
                            row[1] = row[0]
                            cursor.updateRow(row)

                    # Step 3: Delete old field
                    arcpy.DeleteField_management(shapefile, matched)
                    print(f"✅ Renamed '{matched}' to '{correct_name}'")
            else:
                print(f"✅ '{correct_name}' is already correctly named.")
        else:
            print(f"❌ Field for '{correct_name}' not found (even with case mismatch).")


In [4]:
# Process each input shapefile
for key in erase_buffers:
    print(f"\n▶ Processing: {key.upper()}")
    in_fc = erase_buffers[key]
    buffer_fc = outbuffers[key]
    no_overlap_fc = outbuffers_removed[key]

    # Delete existing intermediate layers if needed
    for fc in [buffer_fc, no_overlap_fc]:
        if arcpy.Exists(fc):
            arcpy.Delete_management(fc)

    # Step 1: Buffer
    print("🟦 Creating buffer ...")
    arcpy.Buffer_analysis(
        in_features=in_fc,
        out_feature_class=buffer_fc,
        buffer_distance_or_field=buffer_distance,
        line_side="FULL",
        line_end_type="ROUND",
        dissolve_option="NONE"
    )

    # Step 2: Join original attributes if missing
    print("🔄 Checking & joining attributes ...")
    existing_fields = [f.name for f in arcpy.ListFields(buffer_fc)]
    if id_field not in existing_fields:
        arcpy.JoinField_management(buffer_fc, "ORIG_FID", in_fc, "FID", [id_field])
    else:
        print(f"⚠️ Field '{id_field}' already exists — skipping join.")

    # Step 3: Prepare field list
    fields = [f.name for f in arcpy.ListFields(buffer_fc)
              if f.type not in ("OID", "Geometry")]
    fields.append("SHAPE@")

    field_type = [f.type for f in arcpy.ListFields(buffer_fc) if f.name == id_field][0]

    # Step 4: Create output feature class
    arcpy.CreateFeatureclass_management(
        out_path=out_gdb,
        out_name=no_overlap_fc,
        geometry_type="POLYGON",
        template=buffer_fc
    )

    # Step 5: Erase overlaps feature-by-feature
    with arcpy.da.SearchCursor(buffer_fc, fields) as cursor:
        for row in cursor:
            attr_values = {field: row[i] for i, field in enumerate(fields) if field != "SHAPE@"}
            current_geom = row[-1]

            if field_type in ['Integer', 'SmallInteger', 'Double']:
                where_clause = f"{id_field} <> {attr_values[id_field]}"
                self_clause = f"{id_field} = {attr_values[id_field]}"
            else:
                where_clause = f"{id_field} <> '{attr_values[id_field]}'"
                self_clause = f"{id_field} = '{attr_values[id_field]}'"

            # Create layer for current feature
            arcpy.MakeFeatureLayer_management(buffer_fc, "temp_single_feature", self_clause)
            current_feature_layer = os.path.join("in_memory", f"current_{attr_values[id_field]}")
            arcpy.CopyFeatures_management("temp_single_feature", current_feature_layer)

            # Create layer for all other features
            arcpy.MakeFeatureLayer_management(buffer_fc, "other_buffers", where_clause)

            # Erase overlapping parts
            temp_geom = os.path.join("in_memory", f"temp_geom_{attr_values[id_field]}")
            arcpy.Erase_analysis(current_feature_layer, "other_buffers", temp_geom)

            # Write attributes back
            with arcpy.da.UpdateCursor(temp_geom, fields) as update_cursor:
                for update_row in update_cursor:
                    for i, field in enumerate(fields[:-1]):  # exclude SHAPE@
                        update_row[i] = attr_values[field]
                    update_cursor.updateRow(update_row)

            arcpy.Append_management(temp_geom, no_overlap_fc, "NO_TEST")

            # Clean up temporary layers
            for tmp in ["temp_single_feature", current_feature_layer, temp_geom, "other_buffers"]:
                if arcpy.Exists(tmp):
                    arcpy.Delete_management(tmp)

    print(f"✅ Finished in GDB: {no_overlap_fc}")

    # Step 6: Export final result to shapefile
    shapefile_output = os.path.join(shapefile_output_dir, f"{no_overlap_fc}.shp")
    if arcpy.Exists(shapefile_output):
        arcpy.Delete_management(shapefile_output)
    arcpy.FeatureClassToShapefile_conversion([no_overlap_fc], shapefile_output_dir)
    print(f"📁 Exported shapefile: {shapefile_output}")


▶ Processing: AVG
🟦 Creating buffer ...
🔄 Checking & joining attributes ...
⚠️ Field 'CW_Id' already exists — skipping join.


KeyboardInterrupt: 